Doc: https://www.odoo.com/documentation/17.0/developer/reference/external_api.html

# Connection

For the example, we connect to localhost and with username/password. You may want to connect via an API key, in this case, I invite you to look at the documentation.

Change those parameters to connect to you own db.

### Params

In [10]:
url = "http://localhost:8069"  
db = "db-name"               
username = 'admin'
password = 'admin'

### 1st Connection

To begin with, you have to connect a first time to your db in order to get the uid. The following block has to print "Check uid has a value: 'int'", if it doesn't, it means the connection could not be completed and you should correct it before going any further.

Potential error:
* If your password or username is wrong, uid will be False
* If your url is wrong, the code will crash at step "common.version()" or it will crash afterward
* If you db-name is wrong, you will have a traceback that will say so

In [11]:
import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
common.version()

#authenticate
uid = common.authenticate(db, username, password, {})

print("Check uid has a value:", uid)


Check uid has a value: 155


# Calling methods

We can now interact with the API, by calling methods available into odoo

In [12]:
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

### search_read

__search_read__ is a shortcut for the 2 methods __search__ and __read__, that respectively returns the ids of the records respecting the domain and returns the content of the seleceted fields of the record with the indicated ids.

Note that you also have __search_count__ that counts the number of records respecting the domain and __fields_get__ that inspect a model’s fields

In [13]:
# Example Partner: search and read partners and their names limit = 5 only used for visibility of the example
partners = models.execute_kw(db, uid, password, 'res.partner', 'search_read', [], {'fields': ['name'], 'limit': 5}) 
print("Partners", partners)


# Example Product: search and read products, their name, list and standard prices. For product 
products = models.execute_kw(db, uid, password, 'product.product', 'search_read', [[('detailed_type', 'in', ['consu', 'product'])]], {'fields': ['name', 'list_price', 'standard_price'], 'limit': 5}) 
print("Products", products)

Partners [{'id': 372, 'name': 'AGC Glass Europe'}, {'id': 375, 'name': 'Actiris'}, {'id': 3, 'name': 'Administrator'}, {'id': 374, 'name': 'Ageas Group'}, {'id': 423, 'name': 'Ageas'}]
Products [{'id': 52, 'name': 'ARIS Subscription', 'list_price': 0.0, 'standard_price': 0.0}, {'id': 24, 'name': 'Breakfast/Lunch/Diner', 'list_price': 0.0, 'standard_price': 0.0}, {'id': 54, 'name': 'Hardware', 'list_price': 0.0, 'standard_price': 0.0}, {'id': 26, 'name': 'Hotel/Accomodation', 'list_price': 0.0, 'standard_price': 0.0}, {'id': 53, 'name': 'License Suscription', 'list_price': 0.0, 'standard_price': 0.0}]


### create

__create__ enable to create new records

In [14]:
import datetime

# Example: This code create a so and than returns the its id
so_data = {
    'partner_id': partners[0]['id'], #many2one field, we use the id of the first partner we collected before
    'date_order': datetime.datetime(2026,11,23).strftime('%Y-%m-%d'),
    'state': 'draft',
}
so = models.execute_kw(db, uid, password, 'sale.order', 'create', [so_data])
print("PO number:", so)

PO number: 124


In [15]:
#create the SO lines----------------------------------------------------------------------------------------------------------------------------------------------------
so_line_data = [
    {
        'order_id': so,
        'product_id': products[0]['id']
    },
    {
        'order_id': so,
        'product_id': products[1]['id']
    }
    ]
so_line = models.execute_kw(db, uid, password, 'sale.order.line', 'create', [so_line_data])
print("So lines n°", so_line, "created")

So lines n° [349, 350] created


### write

__write__ enables to edit some existing records

In [16]:
so_before_modif = models.execute_kw(db, uid, password, 'sale.order', 'read', [so], {'fields': ['name', 'state']})
print("before write", so_before_modif)

before write [{'id': 124, 'name': 'S00061', 'state': 'draft'}]


In [17]:
so_modified = models.execute_kw(db, uid, password, 'sale.order', 'write', [[so], {'state': "sale"}])
print(so_modified)

True


In [18]:
so_after_modif = models.execute_kw(db, uid, password, 'sale.order', 'read', [so], {'fields': ['name', 'state']})
print("after write", so_after_modif)

after write [{'id': 124, 'name': 'S00061', 'state': 'sale'}]
